## Cách BatchNorm hoạt động
Cho input $X \in \mathbb{R}^{B \times C \times H \times W}$ với $B,C,H,W$ lần lượt là batch size, channels, height, width. Khi đưa qua BatchNorm, thì $\text{BN}(X)$ được tính bằng cách: \\
- Khởi tạo vector $\mu$ (mean) và $v$ (variance) theo phân phối chuẩn:
$$
\mu, v\in \mathbb{R}^C \space | \space  \mu_i=0, v_i=1 \space | \space \forall \mu_i \in \mu, \forall v_i \in v
$$

- Sau đó $X$ sẽ được biến đối trong không gian bằng 2 phép $\text{normalize}$ và $\text{scale+shift}$

$$
X_{\text{norm}} = \frac{X - \mu}{\sqrt{v + \epsilon}} \\
X_{\text{new}} = X_{\text{norm}} \gamma + \beta
$$

- Trong đó epsilon $\epsilon$ được khởi tạo $=1e^{-5}$ và $\gamma, \beta \in R^C$ hay là vector shape là $(1, C, 1, 1)$. Hay từng feature $X_{\text{new}}[:,i:,:]$ trong $X$ được tính:

$$
X^{\text{new}}_{[:,i:,:]} =  \frac{X_{[:,i,:,:]} - \mu_i}{\sqrt{v + \epsilon}} \gamma + \beta_i
$$

## BatchNorm kết hợp Convolution
Như ta đã biết, tổ hợp phổ biến trong computer vision là $\text{Conv-Relu-BatchNorm}$. BatchNorm diễn ra sau Convolution (ví dụ bỏ qua ReLU) thì làm sao ta kết hợp bước Conv-BatchNorm trong lúc inference để tính toán nhanh hơn.  Gọi $X \in \mathbb{R}^{B, C_{in}, H, W}$ là ma trận đầu vào $W \in \mathbb{W}^{C_{out}, C_{in}, K, K}$ là ma trận trọng số để thực hiện phép convolution và ma trận kết quả là $Y = X * W + b \space | \space Y \in \mathbb{R}^{B, C_{out}, H', W'}$ với $*$ là phép convolution và $b$ là bias.

$$
\text{BN}(Y, \mu, v, \gamma, \beta) = (Y - \mu) \frac{\gamma}{\sqrt{v + \epsilon}} + \beta \\
= (X * W + b - \mu) \frac{\gamma}{\sqrt{v + \epsilon}}. \\
= X * (W \frac{\gamma}{\sqrt{v + \epsilon}}) + \frac{\gamma}{\sqrt{v + \epsilon}} (b - \mu) + \beta
= X * W' + b'
$$

Vậy ma trận trọng số mới là $W' = W \frac{\gamma}{\sqrt{v + \epsilon}}$ và bias mới là $b'= \frac{\gamma}{\sqrt{v + \epsilon}} (b - \mu) + \beta$. \\
Nhưng ở đây ta cần phải lưu ý thêm về chiều khi nhần vào của $W'_{[i,:,:,:]} = \frac{\gamma_i}{\sqrt{v_i + \epsilon}} W_{[i,:,:,:]}$. Đây là lý do ta phải reshape các vector $\gamma, v \in (1, C_{out}, 1, 1)$ thành $(C_{out}), 1, 1, 1)$ trước khi nhân.

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class BatchNorm(nn.Module):
    def __init__(self, num_features, eps=0, momentum=0.1, training_mode=False):
        super().__init__()

        self.training_mode = training_mode
        self.momentum = 0.1
        self.eps = eps

        # trainable parameters
        self.gamma = nn.Parameter(torch.ones(1, num_features, 1, 1))
        self.beta = nn.Parameter(torch.zeros(1, num_features, 1, 1))

        # running mean & variance
        self.r_mean = torch.zeros(1, num_features, 1, 1)
        self.r_var = torch.ones(1, num_features, 1, 1)

    def forward(self, x):
        if self.training_mode:
            x_mean = x.mean([0, 2, 3], keepdim=True)
            x_var = x.var([0, 2, 3], keepdim=True, unbiased=False)

            # Update running mean and variance
            self.r_mean = (1 - self.momentum) * self.r_mean + self.momentum * x_mean
            self.r_var = (1 - self.momentum) * self.r_var + self.momentum * x_var

        else:
            x_mean = self.r_mean
            x_var = self.r_var

        x_norm = (x - x_mean) / torch.sqrt(x_var + self.eps)         # Normalize
        x_out = x_norm * self.gamma + self.beta                      # Scale and Shift
        return x_out

In [7]:
# Khởi tạo ma trận X, trọng số W và bias b
X = torch.randn(12, 32, 224, 224)
W = torch.randn(64, 32, 3, 3)
b = torch.randn(64)

In [8]:
Y = F.conv2d(X, W, b, stride=1, padding=1)
bn = BatchNorm(64)
Z = bn(Y)

In [9]:
# Reshape trước khi nhân
gamma = bn.gamma.view(64, 1, 1, 1)
var = bn.r_var.view(64, 1, 1, 1)
mean = bn.r_mean.view(64, 1, 1, 1)
beta = bn.beta.view(64, 1, 1, 1)
eps = bn.eps

In [10]:
# Reparameterization
W_ = W * (gamma / torch.sqrt(var + eps))
b_ = (gamma / torch.sqrt(var + eps)) * (b.view(64, 1, 1, 1) - mean) + beta
b_ = b_.squeeze()

Z_ = F.conv2d(X, W_, b_, stride=1, padding=1)

In [11]:
# Kiểm tra
print(Z[2,10,56,56])
print(Z_[2,10,56,56])

tensor(-26.9973, grad_fn=<SelectBackward0>)
tensor(-26.9973, grad_fn=<SelectBackward0>)
